# Auto.ria data cleaning


In [1]:
import os
import pandas as pd
import numpy as np

from pathlib import Path

In [2]:
cwd = Path(os.getcwd())
data_path = cwd.parent/'datasets/autoria/autoria_data.csv'
data_path.exists()

False

In [3]:
df = pd.read_csv(data_path, sep='|', encoding='cp1251')
df.info()

FileNotFoundError: [Errno 2] No such file or directory: '/tf/datasets/autoria/autoria_data.csv'

In [ ]:
df.head()

### brand column

It seems like `brand` column can be split into 3:<br> 
`brand` - first part,<br> 
`car model` - everithing between first and last parts and<br> 
`year made` - the last part.

In [ ]:
df['brand'].str.split()

In [ ]:
year_made = df['brand'].str.split().str[-1]
year_made.head()

In [ ]:
pd.to_numeric(year_made).describe()

In [ ]:
df["year_made"] = pd.to_numeric(year_made)
df.head(3)

In [ ]:
# Remove year from the brand column
df["brand"] = df.brand.str.rsplit(n=1).str.get(0)
df.head(3)

In [ ]:
# Split car brand and model into seperate columns
df["model"] = df.brand.str.split(n=1).str.get(1)
df["brand"] = df.brand.str.split(n=1).str.get(0)
df.head(3)

### price and currency columns

It would be best for future modeling to have single currency for price.<br>
So let's convert all prices to USD and remove currency column.

In [ ]:
df['currency'].value_counts()

In [ ]:
# Prices before conversion
df['price'].describe()

In [ ]:
# Convert currencies other than USD to USD
UAH_USD = 0.036
EUR_USD = 1.22

df.loc[df['currency'] == 'UAH', 'price'] *= UAH_USD
df.loc[df['currency'] == 'EUR', 'price'] *= EUR_USD

df['price'] = df['price'].astype(int)

# Prices after conversion
df['price'].describe()

In [ ]:
# Rename price column and remove currency
df.rename(columns={'price': 'price_USD'}, inplace=True)
df.drop(columns=['currency'], inplace=True)
df.head(3)

### mileage column

In [ ]:
df.head(3)

In [ ]:
# Check formats diversity for 'mileage'
df['mileage'].str.split().str.len().value_counts()

In [ ]:
# 'без пробега (no mileage)' must mean that car is fresh 
# and haven't been driven, so it will be reasonable to replace it with zeros.

df.loc[df['mileage'].str.split().str.len() == 2, 'mileage'].value_counts()

In [ ]:
# All non zero mileage values are mesured in 'тис. км (thousand kilometers)'
# So we can just drop that part and leave only numeric one.

df.mileage.str.split(n=1).str[1].value_counts()

In [ ]:
# drop 'тис. км' part
df.loc[df.mileage.str.split().str.len() == 3, 'mileage'] = df.mileage.str.split().str.get(0)
# set to zero all 'без пробега' values
df.loc[df.mileage.str.split().str.len() == 2, 'mileage'] = 0
df['mileage'].head(3)

In [ ]:
# Convert to int
df["mileage"] = pd.to_numeric(df["mileage"])
df["mileage"].head()

In [ ]:
# Rename 'mileage' to 'mileage_kkm' not to forget that values in this column are 1000km
df.rename(columns={'mileage': 'mileage_kkm'}, inplace=True)
df.head(3)

In [ ]:
df['mileage_kkm'].describe()

In [ ]:
df[df['mileage_kkm'] > 1000]

According to [Wikipedia](https://en.wikipedia.org/wiki/Car_longevity):

`typical car lasts closer to 322,000 kilometres`<br>
So it should be safe to assume that mileage over 1,000,000 kilometers is incorrect<br>
(most likely users entered mileage data in plain kilometers)

In [ ]:
# In order to normalize outliers let's divide this values by 1000
df.loc[df['mileage_kkm'] > 1000, 'mileage_kkm'] //= 1000
df['mileage_kkm'].describe()

### fuel_type column

In [ ]:
# 'fuel_type' column can be split into 'fuel_type' and 'engine_size' (engine displacement)
df.head(3)

In [ ]:
df['fuel_type'][2]

In [ ]:
# Remove spaces
df['fuel_type'] = df['fuel_type'].str.strip()
df['fuel_type'][2]

In [ ]:
df['fuel_type'].str.split(',').str.len().value_counts()

In [ ]:
df.loc[df['fuel_type'].str.split(',').str.len() == 1, 'fuel_type']

So it seems like fuel_type column contains 3 kind of values:<br>
only fuel type, only engine size and both(fuel type and engine size)<br>
Let's seperate them

In [ ]:
# Engine size from rows that have both fuel type and engine size
fuel_and_size = df['fuel_type'].str.split(',').str[1]
fuel_and_size.tail(4)

In [ ]:
# Engine size from columns that have only engine size
size_only = df[df['fuel_type'].str.split(',').str[0].str.strip().str.split().str[1] == 'л.'].fuel_type
size_only.head(3)

In [ ]:
fuel_and_size.isna().sum()

In [ ]:
size_only.size

In [ ]:
# Fill NaN's from fuel_and_size with values in size_only
engine_size = fuel_and_size.combine_first(size_only)
engine_size.isna().sum()

In [ ]:
# All engine size values are specified in liters
# hence we can leave only numerical part
engine_size.str.split().str[1].value_counts()

In [ ]:
engine_size = pd.to_numeric(engine_size.str.split().str[0])
engine_size.head(3)

In [ ]:
df['engine_size'] = engine_size
df.tail(5)

In [ ]:
df['engine_size'].describe()

In [ ]:
# This is definitely wrong
df.loc[df['engine_size'] > 10]

In [ ]:
# Set engine_size values bigger than 10 to NaN
# if needed it can be derived from the date
df.loc[df['engine_size'] > 10, 'engine_size'] = np.nan
df['engine_size'].describe()

In [ ]:
# Remove engine size from fuel_type where are both fuel type and engine size
df['fuel_type'] = df['fuel_type'].str.split(',').str[0].str.strip()
df.tail(5)

In [ ]:
df['fuel_type'].value_counts()

In [ ]:
# Set fuel_type column values to NaN where is no fuel type only engine size
df.loc[df['fuel_type'].str.contains('л.', regex=False), 'fuel_type'] = np.nan
df['fuel_type'].value_counts()

In [ ]:
# Normalize category names in fuel_type column

ftype_replace_dict = {
    'fuel_type': {
        'Бензин': 'petrol',
        'Дизель': 'diesel',
        'Газ / Бензин': 'gas/petrol',
        
        'Електро': 'electric',
        'Электро': 'electric',
        
        'Гібрид': 'hybrid',
        'Гибрид': 'hybrid',
        
        'Не вказано': np.nan,
        'Інше': np.nan,
        
        'Газ': 'gas',
        'Газ пропан-бутан': 'gas',
        'Газ метан': 'gas'
    }
}


df.replace(ftype_replace_dict, inplace=True)
df['fuel_type'].value_counts()

### transmission_type column

In [ ]:
df.head(3)

In [ ]:
df['transmission_type'][0]

In [ ]:
# Remove spaces
df['transmission_type'] = df['transmission_type'].str.strip()

In [ ]:
df['transmission_type'][0]

In [ ]:
df['transmission_type'].value_counts()

In [ ]:
# Normalize category names in transmission_type column

ttype_replace_dict = {
    'transmission_type': {
        'Ручна / Механіка': 'manual',
        'Механическая': 'manual',
        
        'Автомат': 'automatic',
        'Не вказано': np.nan,
        
        'Типтронік': 'tiptronic',
        'Типтроник': 'tiptronic',
        
        'Варіатор': 'variator',
        'Вариатор': 'variator',
        
        'Робот': 'robotic',
        'Роботизированная': 'robotic',
        
    }
}

df.replace(ttype_replace_dict, inplace=True)
df['transmission_type'].value_counts()

### pub_date column

In [ ]:
df.head()

In [ ]:
df['pub_date'][0]

In [ ]:
# Remove spaces
df['pub_date'] = df['pub_date'].str.strip()

In [ ]:
df['pub_date'][0]

In [ ]:
# Find inconsistencies in 'pub_date'
df['pub_date'].str.split('.').str.len().value_counts()

In [ ]:
df[df['pub_date'].str.split('.').str.len() == 1]

In [ ]:
# Normalize 'pub_date' values
df.replace({'pub_date': {
    'сьогодні о 12:00': '21.05.2021',
    'Продано': np.nan
}}, inplace=True)

df['pub_date'].str.split('.').str.len().value_counts()

In [ ]:
df['pub_date'] = pd.to_datetime(df['pub_date'])
df.head(3)

### Examine and save cleaned data

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
clean_data_path = data_path.parent/'autoria_clean_data.csv'
df.to_csv(clean_data_path, index=False)

In [ ]:
# Test saves file reads properly
clean_df = pd.read_csv(clean_data_path)
clean_df